In [38]:
# send message size, send message via 5 processes,
# count untill anwer to message size returns
import pickle
import hashlib
import socket
from multiprocessing import Process, Queue
import os

In [39]:
# insert IP of reciever
HOST = '192.168.178.3'
SIZE = 200000000
NUMTHREADS = 20

In [40]:
def init():
    return socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [41]:
def connect(sock, host, port):
    sock.connect((host, port))

In [42]:
# socket is hot untill closed by server
def send(sock, msg):
    sock.sendall(msg)

In [45]:
def start(PORT0, sendFile, times_queue):
    # create hash
    md5_hash = hashlib.md5()
    md5_hash.update(sendFile)
    file_hash = md5_hash.hexdigest()
#     print('file_hash ', file_hash)
    
    # setup socket
    sock = init()
    connect(sock, HOST, PORT0)

    # send len of object
    length = str(len(sendFile))
    send(sock, length.encode('utf-8'))
    
    # wait for answer
    sock.recv(1024)
    
    # send hash value
    send(sock, file_hash.encode('utf-8'))
    
    # wait for answer
    sock.recv(1024)

    # start sending
    send(sock, sendFile)
    
    # wait for duration
    timer = sock.recv(1024).decode('utf8')
    print('timer: ', timer)
    if timer == '-1':
        return
    times_queue.put(timer)

**start sub.run_single() on server to test single connection speed**

In [49]:
times_single = Queue()
bigFile = os.urandom(SIZE)

start(11141, bigFile, times_single)
print(times_single.qsize())

In [50]:
# print single duration
duration = float(times_single.get())
print(SIZE/(duration*1000000), 'MB/s')

**start sub.run() on server to test multiple connection speed**

*can throw some errors, but should keep working*

In [51]:
times_threads = Queue()
size = int(SIZE/NUMTHREADS)
newFile = os.urandom(size)
print(times_threads.qsize())
processes = []
for i in range(NUMTHREADS):
    process = Process(target=start, args=(11111 + i, newFile, times_threads))
    process.start()
    processes.append(process)
for j in processes:
    j.join()
print(times_threads.qsize())
    

In [52]:
print(times_threads.qsize())
duration_threads = 0
counter_threads = times_threads.qsize()
while not times_threads.empty():
    value = float(times_threads.get())
    print(value)
    if value > 0:
        duration_threads += value
print('i was sending with a rate of')
if not duration_threads == 0:
    print((size*counter_threads)/(duration_threads*1000000), 'MB/s')